In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install tabulate
from tabulate import tabulate

### Airline CSV

In [3]:
airlines = pd.read_csv('../Data/airlines.csv')
airlines.tail()

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
6157,21248,GX Airlines,NaN,NaN,CBG,SPRAY,China,Y
6158,21251,Lynx Aviation (L3/SSX),NaN,NaN,SSX,Shasta,United States,N
6159,21268,Jetgo Australia,NaN,JG,\N,NaN,Australia,Y
6160,21270,Air Carnival,NaN,2S,\N,NaN,India,Y
6161,21317,Svyaz Rossiya,Russian Commuter,7R,SJM,RussianConnecty,Russia,Y


In [128]:
airlines.columns

Index(['Airline ID', 'Name_al', 'IATA_al', 'ICAO_al', 'Country_al',
       'Active_al'],
      dtype='object')

In [4]:
#how many rows?
airlines.shape

(6162, 8)

In [5]:
airlines.isnull().sum()

Airline ID       0
Name             0
Alias          506
IATA          4627
ICAO            87
Callsign       808
Country         15
Active           0
dtype: int64

In [6]:
#drop Alias column
airlines.drop('Alias', axis=1, inplace=True)
airlines.drop('Callsign', axis=1, inplace=True)
#airlines.head()

In [7]:
#subset for active airlines
airlines = airlines[airlines['Active']=='Y']
airlines.tail()

,Airline ID,Name,IATA,ICAO,Country,Active
6154,21179,Thai Vietjet Air,NaN,TVJ,Thailand,Y
6157,21248,GX Airlines,NaN,CBG,China,Y
6159,21268,Jetgo Australia,JG,\N,Australia,Y
6160,21270,Air Carnival,2S,\N,India,Y
6161,21317,Svyaz Rossiya,7R,SJM,Russia,Y


In [96]:
airlines.rename(columns = {'Name':'Name_al', 'IATA':'IATA_al', 'ICAO':'ICAO_al','Country':'Country_al', 'Active':'Active_al'}, inplace=True)
airlines.head(2)
#unknown and private, etc., will drop out when joined with delays

,Airline ID,Name_al,IATA_al,ICAO_al,Country_al,Active_al
0,-1,Unknown,-,NaN,\N,Y
1,1,Private flight,-,NaN,NaN,Y


In [90]:
airlines.shape

(1255, 6)

In [91]:
airlines.isnull().sum()

Airline ID      0
Name_al         0
IATA_al       240
ICAO_al        29
Country_al      2
Active_al       0
dtype: int64

In [92]:
#print(tabulate(airlines, headers='keys', tablefmt='pretty'))

In [93]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

In [94]:
#airlines['newname'] = airlines['Name']
#airlines.head(10)

In [95]:
#airlines['newname'] = airlines['newname'].str.replace('-',' ')
#airlines.head()

In [15]:
#airlines['newname'] = airlines['newname'].str.extract(r'(\w+\s+\w+)')
#airlines.head()

### FAA Regions

In [16]:
FAA_Regions = pd.read_csv('../Data/Federal_Aviation_Administration_FAA_Regions.csv')
FAA_Regions.head(11)

,FID,REGION,CNT_REGION,Shape_Leng,Shape_Area
0,1,New England Region,6,152.233742,19.099946
1,2,Central Region,4,51.318016,78.091215
2,3,Southern Region,8,471.324583,94.856190
3,4,Western Pacific Region,4,94.202014,101.933953
4,5,Southwest Region,5,428.811114,138.650729
5,6,Great Lakes Region,8,189.109110,139.986989
6,7,Eastern Region,8,308.913778,48.682795
7,8,Northwest Mountain Region,7,151.363696,197.475811
8,9,Alaska Region,1,1033.192985,278.612678


In [129]:
FAA_Regions.columns

Index(['FID', 'REGION', 'CNT_REGION', 'Shape_Leng', 'Shape_Area'], dtype='object')

### Airports

In [114]:
airports = pd.read_csv('../Data/Airports.csv')
airports.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [115]:
airports.groupby(['Type']).count()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Source
Type,,,,,,,,,,,,,
airport,7698,7698,7649,7698,7698,7698,7698,7698,7698,7698,7698,7698,7698


In [116]:
airports.columns

Index(['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude',
       'Longitude', 'Altitude', 'Timezone', 'DST', 'Tz database time zone',
       'Type', 'Source'],
      dtype='object')

In [117]:
airports.drop(['DST', 'Tz database time zone', 'Type', 'Source'], axis=1, inplace=True)
airports.head(2)

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.20708,145.789001,20,10


In [121]:
airports.rename(columns = {'Name':'Name_ap', 'Country':'Country_ap','IATA':'IATA_ap', 'ICAO':'ICAO_ap', 'Latitude':'Lat_ap', 'Longitude':'Long_ap', 'Altitude':'Alt_ap', 'Timezone':'TZone_ap'}, inplace=True)
airports.head(2)

,Airport ID,Name_ap,City,Country_ap,IATA_ap,ICAO_ap,Lat_ap,Long_ap,Alt_ap,TZone_ap
3212,3411,Barter Island LRRS Airport,Barter Island,United States,BTI,PABA,70.134003,-143.582001,2,-9
3213,3412,Wainwright Air Station,Fort Wainwright,United States,\N,PAWT,70.613403,-159.860001,35,-9


In [122]:
#airports['Country_ap'].unique()

In [123]:
#subset to United States
airports = airports[airports['Country_ap']=='United States']
airports.head()

,Airport ID,Name_ap,City,Country_ap,IATA_ap,ICAO_ap,Lat_ap,Long_ap,Alt_ap,TZone_ap
3212,3411,Barter Island LRRS Airport,Barter Island,United States,BTI,PABA,70.134003,-143.582001,2,-9
3213,3412,Wainwright Air Station,Fort Wainwright,United States,\N,PAWT,70.613403,-159.860001,35,-9
3214,3413,Cape Lisburne LRRS Airport,Cape Lisburne,United States,LUR,PALU,68.875099,-166.110001,16,-9
3215,3414,Point Lay LRRS Airport,Point Lay,United States,PIZ,PPIZ,69.732903,-163.005005,22,-9
3216,3415,Hilo International Airport,Hilo,United States,ITO,PHTO,19.721399,-155.048004,38,-10


In [124]:
airports.isnull().sum()

Airport ID    0
Name_ap       0
City          0
Country_ap    0
IATA_ap       0
ICAO_ap       0
Lat_ap        0
Long_ap       0
Alt_ap        0
TZone_ap      0
dtype: int64

### Passengers

In [126]:
passengers = pd.read_csv('../Data/US Monthly Air Passengers.csv')
passengers.tail(30)

,Sum_PASSENGERS,AIRLINE_ID,CARRIER_NAME,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,YEAR,MONTH
6278790,670,22072.0,WAMOS AIR SA,MIA,"Miami, FL",FL,Florida,US,United States,DPS,"Denpasar, Indonesia",NaN,NaN,ID,Indonesia,2020,4
6278791,596,22072.0,WAMOS AIR SA,MIA,"Miami, FL",FL,Florida,US,United States,DPS,"Denpasar, Indonesia",NaN,NaN,ID,Indonesia,2020,5
6278792,207,22072.0,WAMOS AIR SA,MIA,"Miami, FL",FL,Florida,US,United States,FCO,"Rome, Italy",NaN,NaN,IT,Italy,2020,3
6278793,241,22072.0,WAMOS AIR SA,MIA,"Miami, FL",FL,Florida,US,United States,HRE,"Harare, Zimbabwe",NaN,NaN,ZW,Zimbabwe,2020,6
6278794,269,22072.0,WAMOS AIR SA,MIA,"Miami, FL",FL,Florida,US,United States,LHR,"London, United Kingdom",NaN,NaN,GB,United Kingdom,2020,7
6278795,1337,22072.0,WAMOS AIR SA,MIA,"Miami, FL",FL,Florida,US,United States,MNL,"Manila, Philippines",NaN,NaN,PH,Philippines,2020,3
6278796,1271,22072.0,WAMOS AIR SA,MIA,"Miami, FL",FL,Florida,US,United States,MNL,"Manila, Philippines",NaN,NaN,PH,Philippines,2020,4
6278797,333,22072.0,WAMOS AIR SA,MIA,"Miami, FL",FL,Florida,US,United States,SCL,"Santiago, Chile",NaN,NaN,CL,Chile,2020,6
6278798,163,22072.0,WAMOS AIR SA,MPM,"Maputo, Mozambique",NaN,NaN,MZ,Mozambique,IAD,"Washington, DC",VA,Virginia,US,United States,2020,3
6278799,391,22072.0,WAMOS AIR SA,MSY,"New Orleans, LA",LA,Louisiana,US,United States,DPS,"Denpasar, Indonesia",NaN,NaN,ID,Indonesia,2020,4


In [23]:
passengers.columns

Index(['Sum_PASSENGERS', 'AIRLINE_ID', 'CARRIER_NAME', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM',
       'ORIGIN_COUNTRY', 'ORIGIN_COUNTRY_NAME', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEST_STATE_NM', 'DEST_COUNTRY', 'DEST_COUNTRY_NAME',
       'YEAR', 'MONTH'],
      dtype='object')

In [24]:
passengers.CARRIER_NAME.unique()

array([nan, 'Reeve Aleutian Airways Inc.', 'Northwest Airlines Inc.',
       'Southwest Airlines Co.', 'Aviateca',
       'Canadian Airlines International Ltd.',
       'Guyana Airways Corporation', 'Air Nauru', 'Balair Ag',
       'Sabena Belgian World Air.', 'Air Afrique', 'Air Canada',
       'Compagnie Natl Air France', 'Air-India',
       'National Aviation Company of India Limited d/b/a Air India',
       'Aeromexico', 'Aerolineas Argentinas', 'Royal Air Maroc',
       'Aerovias Nacl De Colombia', 'Finnair Oy',
       'Alitalia-Linee Aeree Italia', 'Compagnia Aerea Italiana',
       'British Airways Plc', 'British Midland Airways Ltd.',
       'British West Indian Airways', 'Caribbean Airlines Limited',
       'Air China', 'China Airlines Ltd.',
       'Cargolux Airlines International S.A', 'Icelandair',
       'Iberia Air Lines Of Spain', 'Japan Air Lines Co. Ltd.',
       'Air Jamaica Limited', 'Korean Air Lines Co. Ltd.',
       'Klm Royal Dutch Airlines', 'Lan-Chile Airlines'

In [25]:
passengers.YEAR.min()

2000

In [26]:
passengers.YEAR.max()

2020

In [27]:
passengers[(passengers['CARRIER_NAME'] == 'Southwest Airlines Co.')]

,Sum_PASSENGERS,AIRLINE_ID,CARRIER_NAME,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,YEAR,MONTH
139227,121,19393.0,Southwest Airlines Co.,ABE,"Allentown/Bethlehem/Easton, PA",PA,Pennsylvania,US,United States,MCO,"Orlando, FL",FL,Florida,US,United States,2005,4
139228,101,19393.0,Southwest Airlines Co.,ABE,"Allentown/Bethlehem/Easton, PA",PA,Pennsylvania,US,United States,MCO,"Orlando, FL",FL,Florida,US,United States,2005,5
139229,134,19393.0,Southwest Airlines Co.,ABE,"Allentown/Bethlehem/Easton, PA",PA,Pennsylvania,US,United States,MCO,"Orlando, FL",FL,Florida,US,United States,2007,3
139230,131,19393.0,Southwest Airlines Co.,ABE,"Allentown/Bethlehem/Easton, PA",PA,Pennsylvania,US,United States,MCO,"Orlando, FL",FL,Florida,US,United States,2007,4
139231,107,19393.0,Southwest Airlines Co.,ABE,"Allentown/Bethlehem/Easton, PA",PA,Pennsylvania,US,United States,TRI,"Bristol/Johnson City/Kingsport, TN",TN,Tennessee,US,United States,2005,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747001,136,19393.0,Southwest Airlines Co.,YNG,"Youngstown/Warren, OH",OH,Ohio,US,United States,PIA,"Peoria, IL",IL,Illinois,US,United States,2015,10
747002,113,19393.0,Southwest Airlines Co.,YNG,"Youngstown/Warren, OH",OH,Ohio,US,United States,ROA,"Roanoke, VA",VA,Virginia,US,United States,2005,9
747003,115,19393.0,Southwest Airlines Co.,YNG,"Youngstown/Warren, OH",OH,Ohio,US,United States,SGF,"Springfield, MO",MO,Missouri,US,United States,2010,10
747004,140,19393.0,Southwest Airlines Co.,YNG,"Youngstown/Warren, OH",OH,Ohio,US,United States,SGF,"Springfield, MO",MO,Missouri,US,United States,2014,10


In [28]:
airlines.isnull().sum()

Airline ID      0
Name_al         0
IATA_al       240
ICAO_al        29
Country_al      2
Active          0
dtype: int64

### Delays - Airline Delay Cause

In [29]:
delays = pd.read_csv('../Data/Airline_Delay_Cause_2010_20.csv')
delays.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2019,12,EV,ExpressJet Airlines LLC,BDL,"Hartford, CT: Bradley International",14.0,4.0,2.00,0.00,...,0.0,2.00,0.0,0.0,114.0,61.0,0.0,0.0,0.0,53.0
1,2019,12,EV,ExpressJet Airlines LLC,BGR,"Bangor, ME: Bangor International",34.0,16.0,4.12,0.00,...,0.0,7.98,2.0,0.0,1450.0,682.0,0.0,231.0,0.0,537.0
2,2019,12,EV,ExpressJet Airlines LLC,BHM,"Birmingham, AL: Birmingham-Shuttlesworth Inter...",65.0,13.0,4.52,0.04,...,0.0,2.48,0.0,0.0,572.0,233.0,2.0,206.0,0.0,131.0
3,2019,12,EV,ExpressJet Airlines LLC,BNA,"Nashville, TN: Nashville International",120.0,41.0,11.01,0.74,...,0.0,15.25,6.0,0.0,2638.0,670.0,164.0,685.0,0.0,1119.0
4,2019,12,EV,ExpressJet Airlines LLC,BRO,"Brownsville, TX: Brownsville South Padre Islan...",96.0,17.0,8.01,0.00,...,0.0,4.33,1.0,0.0,1037.0,578.0,0.0,139.0,0.0,320.0


In [127]:
delays.columns

Index(['year_del', 'month_del', 'al_iata_del', 'carrier_del', 'airport_del',
       'airport_del', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct',
       'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled',
       'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [30]:
delays.rename(columns = {'year':'year_del', 'month':'month_del', 'carrier':'al_iata_del', 'carrier_name':'carrier_del','airport':'airport_del', 'airport_name':'airport_del'}, inplace=True)
delays.head(1)

,year_del,month_del,al_iata_del,carrier_del,airport_del,airport_del,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2019,12,EV,ExpressJet Airlines LLC,BDL,"Hartford, CT: Bradley International",14.0,4.0,2.0,0.0,...,0.0,2.0,0.0,0.0,114.0,61.0,0.0,0.0,0.0,53.0


In [31]:
#delays['new_carrier_name'] = delays['carrier_name']
#delays.head(2)

In [32]:
#delays['new_carrier_name'] = delays['new_carrier_name'].str.replace('-',' ')
#delays.head(2)

In [33]:
#delays['new_carrier_name'] = delays['new_carrier_name'].str.extract(r'(\w+\s+\w+)')
#delays.head(5)

In [34]:
print(sorted(delays['carrier_del'].unique()))

['AirTran Airways Corporation', 'Alaska Airlines Inc.', 'Allegiant Air', 'American Airlines Inc.', 'American Eagle Airlines Inc.', 'Atlantic Southeast Airlines', 'Comair Inc.', 'Continental Air Lines Inc.', 'Delta Air Lines Inc.', 'Endeavor Air Inc.', 'Envoy Air', 'ExpressJet Airlines Inc.', 'ExpressJet Airlines LLC', 'Frontier Airlines Inc.', 'Hawaiian Airlines Inc.', 'JetBlue Airways', 'Mesa Airlines Inc.', 'PSA Airlines Inc.', 'Pinnacle Airlines Inc.', 'Republic Airline', 'SkyWest Airlines Inc.', 'Southwest Airlines Co.', 'Spirit Air Lines', 'US Airways Inc.', 'United Air Lines Inc.', 'Virgin America']


In [35]:
delays.columns

Index(['year_del', 'month_del', 'al_iata_del', 'carrier_del', 'airport_del',
       'airport_del', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct',
       'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled',
       'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay'],
      dtype='object')

### Airplanes

In [36]:
airplanes = pd.read_csv('../Data/airplanes.csv')
airplanes.head()

,Name,IATA code,ICAO code
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [37]:
airplanes.rename(columns = {'Name':'Plane', 'IATA code':'Plane_IATA','ICAO code':'Plane_ICAO'}, inplace=True)
airplanes.head()

,Plane,Plane_IATA,Plane_ICAO
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [38]:
airplanes.isnull().sum()

Plane         0
Plane_IATA    0
Plane_ICAO    0
dtype: int64

### Routes

In [39]:
routes = pd.read_csv('../Data/routes.csv')
routes.head()

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


In [40]:
routes.shape

(67663, 9)

In [41]:
routes.isnull().sum()

Airline                       0
Airline ID                    0
Source airport                0
Source airport ID             0
Destination airport           0
Destination airport ID        0
Codeshare                 53066
Stops                         0
Equipment                    18
dtype: int64

In [42]:
routes.drop('Codeshare', axis=1, inplace=True)
routes.head(1)

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,0,CR2


In [43]:
routes.rename(columns = {'Airline':'Airline_IATA_r', 'Airline ID':'Airline_ID_r','Stops':'Stops_r','Equipment':'Plane_IATA_r'}, inplace=True)
routes.head(1)

,Airline_IATA_r,Airline_ID_r,Source airport,Source airport ID,Destination airport,Destination airport ID,Stops_r,Plane_IATA_r
0,2B,410,AER,2965,KZN,2990,0,CR2


In [44]:
### Finding Number of Carrier Delays by more than 15 minutes
### Join Airlines with Delays

In [45]:
delays.head(2)

,year_del,month_del,al_iata_del,carrier_del,airport_del,airport_del,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2019,12,EV,ExpressJet Airlines LLC,BDL,"Hartford, CT: Bradley International",14.0,4.0,2.00,0.0,...,0.0,2.00,0.0,0.0,114.0,61.0,0.0,0.0,0.0,53.0
1,2019,12,EV,ExpressJet Airlines LLC,BGR,"Bangor, ME: Bangor International",34.0,16.0,4.12,0.0,...,0.0,7.98,2.0,0.0,1450.0,682.0,0.0,231.0,0.0,537.0


In [46]:
delays.carrier_del.unique()

array(['ExpressJet Airlines LLC', 'Frontier Airlines Inc.',
       'Allegiant Air', 'Hawaiian Airlines Inc.', 'Envoy Air',
       'Spirit Air Lines', 'Southwest Airlines Co.', 'Mesa Airlines Inc.',
       'Republic Airline', 'PSA Airlines Inc.', 'SkyWest Airlines Inc.',
       'United Air Lines Inc.', 'Endeavor Air Inc.',
       'American Airlines Inc.', 'Alaska Airlines Inc.',
       'JetBlue Airways', 'Delta Air Lines Inc.',
       'ExpressJet Airlines Inc.', 'Virgin America', 'US Airways Inc.',
       'AirTran Airways Corporation', 'American Eagle Airlines Inc.',
       'Pinnacle Airlines Inc.', 'Atlantic Southeast Airlines',
       'Continental Air Lines Inc.', 'Comair Inc.'], dtype=object)

In [47]:
delays.columns

Index(['year_del', 'month_del', 'al_iata_del', 'carrier_del', 'airport_del',
       'airport_del', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct',
       'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled',
       'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [48]:
airlines.Name_al.unique()

array(['Unknown', 'Private flight', '1Time Airline', ...,
       'Jetgo Australia', 'Air Carnival', 'Svyaz Rossiya'], dtype=object)

In [49]:
airlines.head()

,Airline ID,Name_al,IATA_al,ICAO_al,Country_al,Active
0,-1,Unknown,-,NaN,\N,Y
1,1,Private flight,-,NaN,NaN,Y
3,3,1Time Airline,1T,RNX,South Africa,Y
10,10,40-Mile Air,Q5,MLA,United States,Y
13,13,Ansett Australia,AN,AAA,Australia,Y


In [50]:
delays.isnull().sum()

year_del                 0
month_del                0
al_iata_del              0
carrier_del              0
airport_del              0
airport_del              0
arr_flights            162
arr_del15              195
carrier_ct             162
weather_ct             162
nas_ct                 162
security_ct            162
late_aircraft_ct       162
arr_cancelled          162
arr_diverted           162
arr_delay              162
carrier_delay          162
weather_delay          162
nas_delay              162
security_delay         162
late_aircraft_delay    162
dtype: int64

In [51]:
delays.columns

Index(['year_del', 'month_del', 'al_iata_del', 'carrier_del', 'airport_del',
       'airport_del', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct',
       'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled',
       'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [52]:
airlines.columns

Index(['Airline ID', 'Name_al', 'IATA_al', 'ICAO_al', 'Country_al', 'Active'], dtype='object')

### Merge Airline dataset

In [53]:
#mergedStuff = pd.merge(df1, df2, on=['Name'], how='inner')
delays_air = delays.merge(airlines, left_on='al_iata_del', right_on='IATA_al', how='left')
delays_air.head(2)

,year_del,month_del,al_iata_del,carrier_del,airport_del,airport_del,arr_flights,arr_del15,carrier_ct,weather_ct,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,Airline ID,Name_al,IATA_al,ICAO_al,Country_al,Active
0,2019,12,EV,ExpressJet Airlines LLC,BDL,"Hartford, CT: Bradley International",14.0,4.0,2.00,0.0,...,0.0,0.0,0.0,53.0,452,Atlantic Southeast Airlines,EV,ASQ,United States,Y
1,2019,12,EV,ExpressJet Airlines LLC,BGR,"Bangor, ME: Bangor International",34.0,16.0,4.12,0.0,...,0.0,231.0,0.0,537.0,452,Atlantic Southeast Airlines,EV,ASQ,United States,Y


In [86]:
delays_air.columns

Index(['year_del', 'month_del', 'al_iata_del', 'carrier_del', 'airport_del',
       'airport_del', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct',
       'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled',
       'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay', 'Airline ID',
       'Name_al', 'IATA_al', 'ICAO_al', 'Country_al', 'Active'],
      dtype='object')

In [88]:
delays_air = delays_air.loc[:, ['year_del', 'month_del', 'al_iata_del', 'carrier_del', 'airport_del', 'airport_del', 'Airline ID', 'Name_al', 'IATA_al', 'ICAO_al', 'Country_al', 'Active', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct',
       'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled','arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay']]

In [54]:
#can also use iterrows
#for x in airlines.itertuples():
#    if x[2].find('ExpressJet') != -1:
#        print(x)

In [55]:
#thinking about joining on airline name after filter but expressjet appears to be the only one with a history of changes in delays data
#express = airlines[airlines['Name'].str.contains('ExpressJet')]
#express.head()

In [66]:
delays_airlines = delays_air.groupby(['carrier_del'])['arr_del15'].sum().sort_values(ascending=False).reset_index(name='sum_delays')
delays_airlines.head(2)

,carrier_del,sum_delays
0,Southwest Airlines Co.,2375572.0
1,American Airlines Inc.,1385103.0


In [67]:
sum_arr_air = delays_air.groupby(['carrier_del'])['arr_flights'].sum().sort_values(ascending=False).reset_index(name='sum_arrivals')
sum_arr_air.head(2)

,carrier_del,sum_arrivals
0,Southwest Airlines Co.,12333317.0
1,Delta Air Lines Inc.,8410684.0


In [71]:
delays_all_airlines = delays_airlines.merge(sum_arr_air, left_on='carrier_del', right_on='carrier_del', how='left')
delays_all_airlines.head()

,carrier_del,sum_delays,sum_arrivals
0,Southwest Airlines Co.,2375572.0,12333317.0
1,American Airlines Inc.,1385103.0,7080371.0
2,Delta Air Lines Inc.,1230207.0,8410684.0
3,SkyWest Airlines Inc.,1186901.0,6555532.0
4,ExpressJet Airlines Inc.,975862.0,4616917.0


### -----------------------------------------------------------------------------

### Top/Bottom Airline Delay Percentage from 2010-2019

In [72]:
delays_all_airlines['pct_delay'] = delays_all_airlines['sum_delays'] / delays_all_airlines['sum_arrivals']
delays_all_airlines = delays_all_airlines.sort_values(by='pct_delay', ascending=False)
delays_all_airlines.tail(10)

,carrier_del,sum_delays,sum_arrivals,pct_delay
20,Pinnacle Airlines Inc.,81394.0,438115.0,0.185782
17,Republic Airline,118606.0,645239.0,0.183817
3,SkyWest Airlines Inc.,1186901.0,6555532.0,0.181053
13,Mesa Airlines Inc.,182474.0,1049018.0,0.173947
19,Endeavor Air Inc.,105408.0,622999.0,0.169194
8,US Airways Inc.,370891.0,2246049.0,0.165130
15,AirTran Airways Corporation,148611.0,969156.0,0.153341
2,Delta Air Lines Inc.,1230207.0,8410684.0,0.146267
10,Alaska Airlines Inc.,250481.0,1788691.0,0.140036
22,Hawaiian Airlines Inc.,67564.0,756363.0,0.089327


In [75]:
delays.year_del.min()

2010

In [76]:
delays.year_del.max()

2019

# -----------------------------------------------------------

### Average & Mean Delays for All Airlines

In [77]:
delays_all_airlines.sum_delays.sum() / delays_all_airlines.sum_arrivals.sum()

0.18561005370682257

In [79]:
delays_all_airlines.pct_delay.mean()

0.19093195931715773

### --------------------------------------------------------------------------------

In [80]:
delays_air.head(1)

,year_del,month_del,al_iata_del,carrier_del,airport_del,airport_del,arr_flights,arr_del15,carrier_ct,weather_ct,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,Airline ID,Name_al,IATA_al,ICAO_al,Country_al,Active
0,2019,12,EV,ExpressJet Airlines LLC,BDL,"Hartford, CT: Bradley International",14.0,4.0,2.0,0.0,...,0.0,0.0,0.0,53.0,452,Atlantic Southeast Airlines,EV,ASQ,United States,Y


In [81]:
delays_air.rename(columns = {'iata_code':'carrier_code'}, inplace=True)
delays_air.rename(columns = {'airport':'airport_code_del'}, inplace=True)
delays_air.rename(columns = {'airport_name':'airport_name_del'}, inplace = True)
delays_air.head(1)

,year_del,month_del,al_iata_del,carrier_del,airport_del,airport_del,arr_flights,arr_del15,carrier_ct,weather_ct,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,Airline ID,Name_al,IATA_al,ICAO_al,Country_al,Active
0,2019,12,EV,ExpressJet Airlines LLC,BDL,"Hartford, CT: Bradley International",14.0,4.0,2.0,0.0,...,0.0,0.0,0.0,53.0,452,Atlantic Southeast Airlines,EV,ASQ,United States,Y


In [82]:
delays_air.columns

Index(['year_del', 'month_del', 'al_iata_del', 'carrier_del', 'airport_del',
       'airport_del', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct',
       'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled',
       'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay', 'Airline ID',
       'Name_al', 'IATA_al', 'ICAO_al', 'Country_al', 'Active'],
      dtype='object')

In [83]:
#column_names = ["C", "A", "B"]


#df = df.reindex(columns=column_names)

In [85]:
#finding month of most airline delays of 15 minutes or more
delays_air.groupby(['year_del', 'month_del', 'carrier_del','arr_flights', 'arr_del15'])['arr_del15'].sum().sort_values(ascending=False).reset_index(name='sum_delays')

,year_del,month_del,carrier_del,arr_flights,arr_del15,sum_delays
0,2013,7,Delta Air Lines Inc.,18649.0,5268.0,5268.0
1,2013,6,Delta Air Lines Inc.,18144.0,4395.0,4395.0
2,2018,6,Delta Air Lines Inc.,21029.0,4176.0,4176.0
3,2014,1,Delta Air Lines Inc.,17071.0,4024.0,4024.0
4,2012,9,American Airlines Inc.,12360.0,3919.0,3919.0
...,...,...,...,...,...,...
150315,2019,11,Frontier Airlines Inc.,5.0,0.0,0.0
150316,2019,11,Frontier Airlines Inc.,6.0,0.0,0.0
150317,2018,5,Republic Airline,2.0,0.0,0.0
150318,2018,5,Republic Airline,5.0,0.0,0.0
